In [1]:
from pathlib import Path
import pandas as pd

---
# Внешние факторы

In [2]:
# 4 page excel
file = pd.ExcelFile('data/raw/Внешние факторы.xlsx')
sheets = file.sheet_names

# orders count

In [3]:
df = pd.read_excel(file, sheets[0], header=None)
print(df.shape)
df.head()

(2122, 69)


,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,На этом листе приведены ЗАКАЗЫ по дате создани...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Иерархия - ГодКварталМесяцДата,(несколько элементов),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Заказы - Текущий статус,(несколько элементов),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# cut top nan rows and last row with summaries
df = df.iloc[7:-1]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
7,NaN,Moscow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saint Petersburg Итог,NaN,NaN
8,Календарь - Дата,NaN,fit,power,daily,daily plus,balance,unifood,vega,super_fit,...,m_fit,cv_light,cv_max,gf_adidas,bf_pack,basic,priem_plus,NaN,NaN,NaN
9,2017-01-01,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,4
10,2017-01-03,NaN,144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147,291,291
11,2017-01-04,NaN,NaN,180,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129,309,309


In [5]:
root = Path('data/processed/')

## msk

In [6]:
def flat_table(table: pd.DataFrame, city_id: int) -> pd.DataFrame:
    # makes table (date, product_name, city_id) from (date, product1, ... product_i)
    dfs = []

    cols = table.columns.tolist()[1:]
    for col in cols:
        d = table[['date', col]].rename({col: 'count'}, axis=1)
        fvi = d['count'].first_valid_index()
        d = d.iloc[fvi:].fillna(0)
        d = d.assign(product_name=col)
        
        if len(d) == 0:
            continue
        dfs.append(d)
    
    df = pd.concat(dfs, axis=0)
    df = df.assign(city_id=city_id)
    return df

In [7]:
df = df.drop(1, axis=1)
# cut products data
orders_count = df.iloc[1:, :30]

# make header
headers = orders_count.iloc[0, :].tolist()
headers[0] = 'date'
orders_count.columns = headers

orders_count_msk = orders_count.iloc[1:].reset_index(drop=True)

## spb

In [8]:
dates = orders_count.date

# cut products data
orders_count = df.iloc[1:, 33:-3]

# make header
headers = orders_count.iloc[0, :].tolist()
orders_count.columns = headers

orders_count = orders_count.iloc[1:]
orders_count_spb = pd.concat((dates, orders_count), axis=1)

In [9]:
msk = flat_table(orders_count_msk, city_id=2)
spb = flat_table(orders_count_spb, city_id=1)

df = pd.concat((msk, spb))
df.to_csv(root / 'orders_count.csv', index=False)

# new orders count

In [10]:
df = pd.read_excel(file, sheets[1], header=None)
print(df.shape)
df.head()

(2114, 65)


,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,Мы классифицируем ЗАКАЗЫ на НОВЫЕ и АКТИВНЫЕ. ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Иерархия - ГодКварталМесяцДата,(несколько элементов),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Заказы - Текущий статус,(несколько элементов),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Заказы - Жизненный цикл заказа (Новый\Активный...,Новый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# cut top nan rows and last row with summaries
df = df.iloc[7:-1]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
7,NaN,Moscow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saint Petersburg Итог,NaN,NaN
8,Календарь - Дата,NaN,fit,power,daily,daily plus,balance,unifood,vega,super_fit,...,m_fit,cv_light,cv_max,gf_adidas,bf_pack,basic,priem_plus,NaN,NaN,NaN
9,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
10,2017-01-03,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,34,34
11,2017-01-04,NaN,NaN,33,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,50,50


## msk

In [12]:
df = df.drop(1, axis=1)
# cut products data
orders_count = df.iloc[1:, :30]

# make header
headers = orders_count.iloc[0, :].tolist()
headers[0] = 'date'
orders_count.columns = headers

orders_count_msk = orders_count.iloc[1:].reset_index(drop=True)

## spb

In [13]:
# cut products data
orders_count = df.iloc[1:, 33:-3]

# make header
headers = orders_count.iloc[0, :].tolist()
orders_count.columns = headers

orders_count = orders_count.iloc[1:]
orders_count_spb = pd.concat((orders_count_msk.date, orders_count), axis=1).reset_index(drop=True)

In [14]:
msk = flat_table(orders_count_msk, city_id=2)
spb = flat_table(orders_count_spb, city_id=1)

df = pd.concat((msk, spb))
df.to_csv(root / 'new_orders_count.csv', index=False)

# discounts

In [15]:
df = pd.read_excel(file, sheets[2], header=None)
print(df.shape)
df.head()

(2121, 68)


,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,% Скидок -- это численное выражение маркетинго...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Иерархия - ГодКварталМесяцДата,(несколько элементов),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Доставка - Статусы - Текущий статус,доставлен,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# cut top nan rows and last row with summaries
df = df.iloc[7:-1]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
7,NaN,Moscow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saint Petersburg Итог,NaN,NaN
8,Календарь - Дата,NaN,fit,power,daily,daily plus,balance,unifood,vega,super_fit,...,m_fit,cv_light,cv_max,gf_adidas,bf_pack,basic,priem_plus,NaN,NaN,NaN
9,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121951,0.121951,0.121951
10,2017-01-03,NaN,-0.399737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.240751,-0.323358,-0.323358
11,2017-01-04,NaN,-2.377473,-0.500152,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.240305,-0.399726,-0.399726


## msk

In [17]:
df = df.drop(1, axis=1)
# cut products data
orders_count = df.iloc[1:, :30]

# make header
headers = orders_count.iloc[0, :].tolist()
headers[0] = 'date'
orders_count.columns = headers

orders_count_msk = orders_count.iloc[1:].reset_index(drop=True)

## spb

In [18]:
# cut products data
orders_count = df.iloc[1:, 33:-3]

# make header
headers = orders_count.iloc[0, :].tolist()
orders_count.columns = headers

orders_count = orders_count.iloc[1:]
orders_count_spb = pd.concat((orders_count_msk.date, orders_count), axis=1).reset_index(drop=True)

In [19]:
msk = flat_table(orders_count_msk, city_id=2)
spb = flat_table(orders_count_spb, city_id=1)

df = pd.concat((msk, spb))
df.to_csv(root / 'discounts.csv', index=False)

# custom order rate

In [20]:
df = pd.read_excel(file, sheets[3], header=None)
print(df.shape)
df.head()

(1418, 31)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,Линейки ГроуФуд содержат стандартное заранее и...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Иерархия - ГодКварталМесяцДата,(несколько элементов),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Доставка - Статусы - Текущий статус,доставлен,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# cut top nan rows and last row with summaries
df = df.iloc[7:-1]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
7,NaN,Moscow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saint Petersburg Итог,NaN,NaN
8,Календарь - Дата,fit,power,daily,balance,super_fit,breakfasts_2x,priem,fit_express,dinner,...,samokat,samokat_7,perekrestok,rosneft,m_fit,gf_adidas,bf_pack,NaN,NaN,NaN
9,2018-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010638,0.004044,0.004044
10,2018-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001574,0.000611,0.000611
11,2018-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003463,0.001303,0.001303


## msk

In [22]:
df = df.drop(1, axis=1)
# cut products data
orders_count = df.iloc[1:, :13]

# make header
headers = orders_count.iloc[0, :].tolist()
headers[0] = 'date'
orders_count.columns = headers

orders_count_msk = orders_count.iloc[1:].reset_index(drop=True)

## spb

In [23]:
# cut products data
orders_count = df.iloc[1:, 14:28]

# make header
headers = orders_count.iloc[0, :].tolist()
orders_count.columns = headers

orders_count = orders_count.iloc[1:]
orders_count_spb = pd.concat((orders_count_msk.date, orders_count), axis=1).reset_index(drop=True)

In [24]:
orders_count_msk

,date,power,daily,balance,super_fit,breakfasts_2x,priem,fit_express,dinner,m_fit,gf_adidas,bf_pack,NaN
0,2018-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001106
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,2022-10-22,0.523529,0.546296,0.493197,0.569024,0.6,NaN,0.690789,NaN,0.409962,NaN,NaN,0.368403
1404,2022-10-23,0.546939,0.543391,0.501188,0.526316,0.731707,NaN,0.704453,NaN,0.479121,NaN,NaN,0.402842
1405,2022-10-24,0.533937,0.541735,0.478836,0.576419,0.571429,NaN,0.726087,NaN,0.418848,NaN,NaN,0.384017
1406,2022-10-25,0.552174,0.522639,0.5311,0.553459,0.791667,NaN,0.686441,NaN,0.481069,NaN,NaN,0.413645


In [25]:
msk = flat_table(orders_count_msk, city_id=2)
spb = flat_table(orders_count_spb, city_id=1)

df = pd.concat((msk, spb))
df.to_csv(root / 'custom_order_rate.csv', index=False)

---
# Фактическое_число_доставок_по_линейкам_и_городам

In [26]:
df = pd.read_excel('data/raw/Фактическое_число_доставок_по_линейкам_и_городам.xlsx')
print(df.shape)
df.head()

(36206, 5)


,count,planned_delivery_date,city_id,production_type_id,name
0,34,2022-10-27,2,185,elementaree_1
1,168,2022-10-26,1,19,balance_2_2
2,1,2022-10-26,1,20,detox_2_2
3,104,2022-10-26,1,21,power_2_2
4,172,2022-10-26,1,22,superfit_2_2


In [27]:
df = df[df.city_id.isin((2, 1))]

In [28]:
df.to_csv(root / 'fact_deliveries_count.csv', index=False)